In [31]:
import librosa
import soundfile
import numpy as np
import os, pickle,glob
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import Sequential

In [32]:
path= "/content/drive/My Drive/dataset"

In [33]:
#extracting features from audio file
def extract_features(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as s_file:
        x=s_file.read(dtype="float32")
        sample_rate=s_file.samplerate

        if chroma:
            stft=np.abs(librosa.stft(x))
        result=np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

In [34]:
#Spliting train-test
def train_test(test_size=0.2):
    x=[]
    y=[]
    for file in glob.glob(r"/content/drive/My Drive/dataset//Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=file_name.split('-')[2]
        if(emotion=='01' or emotion=='02'  or  emotion=='06'):
            emotion=0
        elif(emotion=='03'):
            emotion=1
        if emotion==0 or emotion==1:

            feature=extract_features(file,mfcc=True,chroma=True,mel=True)
            x.append(feature)
            y.append(emotion)

    return train_test_split(np.array(x),y,test_size=test_size,random_state=9)

In [40]:
x_train,x_test,y_train,y_test=train_test(test_size=0.25)

In [41]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [42]:
y_pred=model.predict(x_test)
print(y_pred)


[1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1
 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 1 0 1 1 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0
 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0]


In [43]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [44]:
print(accuracy)

0.875
